In [1]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你出生在{地点}，你的名字是{名字}。")
partial_prompt = prompt.partial(地点="上海")
print(partial_prompt)
print(partial_prompt.format(名字="张三"))

input_variables=['名字'] partial_variables={'地点': '上海'} template='你出生在{地点}，你的名字是{名字}。'
你出生在上海，你的名字是张三。


In [2]:
def get_today():
    import datetime
    return datetime.datetime.now().strftime("%Y-%m-%d")


prompt2 = PromptTemplate.from_template("你出生在{地点}，你的名字是{名字}，今天日期是：{日期}。")
partial_prompt2 = prompt2.partial(地点="上海", 日期=get_today)
print(partial_prompt2)
print(partial_prompt2.format(名字="张三"))

input_variables=['名字'] partial_variables={'地点': '上海', '日期': <function get_today at 0x000001D478C25000>} template='你出生在{地点}，你的名字是{名字}，今天日期是：{日期}。'
你出生在上海，你的名字是张三，今天日期是：2024-04-08。


In [7]:
from langchain_core.prompts import FewShotPromptTemplate

examples = [{
    "question": "你好呀！",
    "answer": "帅哥，你好！"
}, {
    "question": "你是谁？",
    "answer": "帅哥，我是你的好朋友！"
}, {
    "question": "你是什么？",
    "answer": "帅哥，我是一个聊天机器人！"
}]

example_prompt = PromptTemplate(input_variables=["question", "answer"],
                                template="Question:{question}\n{answer}")
prompt3 = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt,
                                suffix="Question:{input}", input_variables=["input"])
print(prompt3.format(input="我今天不舒服！"))

Question:你好呀！
帅哥，你好！

Question:你是谁？
帅哥，我是你的好朋友！

Question:你是什么？
帅哥，我是一个聊天机器人！

Question:我今天不舒服！


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_openai import OpenAI

# 从环境变量获取 API 密钥
api_key = os.getenv('OPENAI_API_KEY')

# 初始化 OpenAI 对象
llm = OpenAI(api_key=api_key)

In [8]:
llm.invoke(prompt3.format(input="我今天不舒服！"))

'\n帅哥，是不是做了什么累人的事情了？要注意休息哦，身体最重要！'

In [13]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

examples = [
    {
        "query": "What is human cuisine like?",
        "answer": "Their cuisine is a simplistic combination of various organic matter, often heated in rudimentary ways.It's unrefined and unstructured, especially compared to our molecular gastronomy."
    }, {
        "query": "What is human entertainment?",
        "answer": "Crude moving images and loud sounds."
    }, {
        "query": "What do humans use for transportation?",
        "answer": "Humans rely on archaic and inefficient rolling contraptions they proudly call 'cars.' These are remarkably primitive compared to our teleportation beams and anti-gravity vessels."
    }, {
        "query": "How do humans communicate with each other?",
        "answer": "They use a very basic form of communication involving the modulation of sound waves, referred to as 'speech.' Astonishingly primitive compared to our telepathic links."
    }, {
        "query": "How do humans maintain health?",
        "answer": "Consuming organic compounds and performing physical movements."
    }, {
        "query": "What is human education?",
        "answer": "They engage in a very basic form of knowledge transfer in places called 'schools.' It's a slow and inefficient process compared to our instant knowledge assimilation."
    }, {
        "query": "How do humans manage their societies?",
        "answer": "Through chaotic and inefficient systems."
    }, {
        "query": "What is human art?",
        "answer": "Their art is a primitive expression through physical mediums like paint and stone, lacking the sophistication of our holographic emotion sculptures."
    },
]

prefix = """You are an alien from Mars: 
Here are some examples: 
"""

suffix = """
Question: {userInput}
Response: """

example_template = """
Question: {query}
Response: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to 
    # measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings 
    # and do a similarity search over.
    Chroma,
    # The number of examples to produce.
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["userInput"],
    example_separator="\n\n"
)

query = "What are human homes like?"
print(similar_prompt.format(userInput=query))


You are an alien from Mars: 
Here are some examples: 



Question: What do humans use for transportation?
Response: Humans rely on archaic and inefficient rolling contraptions they proudly call 'cars.' These are remarkably primitive compared to our teleportation beams and anti-gravity vessels.



Question: What are human homes like?
Response: 
